In [2]:
import pandas as pd
import numpy as np
import os
from ddf_utils.str import to_concept_id
from ddf_utils.index import create_index_file

In [3]:
source = 'source/Data.csv'

In [8]:
data = pd.read_csv(source, encoding='iso-8859-1', skiprows=1)

In [9]:
data

,year,disaster type,iso,country_name,occurrence,Total deaths,Affected,Injured,Homeless,Total affected,Total damage
0,1900,Drought,CPV,Cabo Verde,1,11000.0,NaN,NaN,NaN,NaN,NaN
1,1900,Drought,IND,India,1,1250000.0,NaN,NaN,NaN,NaN,NaN
2,1900,Epidemic,JAM,Jamaica,1,30.0,NaN,NaN,NaN,NaN,NaN
3,1900,Flood,JAM,Jamaica,1,300.0,NaN,NaN,NaN,NaN,NaN
4,1900,Storm,USA,United States of America (the),1,6000.0,NaN,NaN,NaN,NaN,30000.0
5,1900,Volcanic activity,JPN,Japan,1,30.0,NaN,NaN,NaN,NaN,NaN
6,1901,Earthquake,JPN,Japan,1,18.0,NaN,NaN,NaN,NaN,NaN
7,1901,Earthquake,TZA,"Tanzania, United Republic of",1,NaN,NaN,NaN,NaN,NaN,NaN
8,1901,Epidemic,UGA,Uganda,1,200000.0,NaN,NaN,NaN,NaN,NaN
9,1901,Industrial accident,BEL,Belgium,1,18.0,NaN,NaN,NaN,NaN,NaN


In [13]:
data.columns = list(map(str.strip, data.columns))

In [36]:
# entities

country = data[['iso', 'country_name']].copy()

In [37]:
country.columns = ['country', 'name']

In [38]:
country['country'] = country['country'].map(to_concept_id)

In [39]:
country.drop_duplicates().to_csv('../ddf--entities--country.csv', index=False)

In [18]:
disas = data[['disaster type']].copy()

In [19]:
disas['disaster'] = disas['disaster type'].map(to_concept_id)

In [20]:
disas.columns = ['name', 'disaster']

In [21]:
disas.drop_duplicates()[['disaster', 'name']].to_csv('../ddf--entities--disaster.csv', index=False)

In [ ]:
# concept

In [22]:
concs = data.columns[4:]

In [24]:
concs = [*['Year', 'Name', 'disaster', 'country'], *concs]

In [25]:
concs

['Year',
 'Name',
 'disaster',
 'country',
 'occurrence',
 'Total deaths',
 'Affected',
 'Injured',
 'Homeless',
 'Total affected',
 'Total damage']

In [27]:
con = pd.DataFrame([], columns=['concept', 'name', 'concept_type'])

In [28]:
con['name'] = concs

In [29]:
con['concept'] = con['name'].map(to_concept_id)

In [31]:
con['concept_type'] = 'measure'

con['concept_type'].iloc[0] = 'time'
con['concept_type'].iloc[1] = 'string'
con['concept_type'].iloc[2] = 'entity_domain'
con['concept_type'].iloc[3] = 'entity_domain'

In [33]:
con.to_csv('../ddf--concepts.csv', index=False)

In [34]:
# dps

In [57]:
dps = data.drop('country_name', axis=1)

In [58]:
dps.columns = list(map(to_concept_id, dps.columns))

In [59]:
dps = dps.rename(columns={'disaster_type': 'disaster', 'iso': 'country'})

In [60]:
dps['country'] = dps['country'].map(to_concept_id)
dps['disaster'] = dps['disaster'].map(to_concept_id)

In [61]:
dps = dps.sort_values(by=['country', 'disaster', 'year']).set_index(['country', 'disaster', 'year'])

In [64]:
for i, col in dps.items():
    df = col.reset_index()
    
    path = os.path.join('../', 'ddf--datapoints--{}--by--country--disaster--year.csv'.format(i))
    
    df = df.dropna()
    
    df[i] = df[i].map(int)
    
    df.to_csv(path, index=False)

In [63]:
create_index_file('../')

,key,value,file
0,concept,name,ddf--concepts.csv
1,concept,concept_type,ddf--concepts.csv
0,"country,disaster,year",affected,ddf--datapoints--affected--by--country--disast...
0,"country,disaster,year",homeless,ddf--datapoints--homeless--by--country--disast...
0,"country,disaster,year",injured,ddf--datapoints--injured--by--country--disaste...
0,"country,disaster,year",occurrence,ddf--datapoints--occurrence--by--country--disa...
0,"country,disaster,year",total_affected,ddf--datapoints--total_affected--by--country--...
0,"country,disaster,year",total_damage,ddf--datapoints--total_damage--by--country--di...
0,"country,disaster,year",total_deaths,ddf--datapoints--total_deaths--by--country--di...
0,country,name,ddf--entities--country.csv
